In [12]:


#IMPORTS


import keras
import numpy as np
from keras.models import Sequential
import os
import shutil
from PIL import Image
import distutils
from distutils.dir_util import copy_tree

import  sys
from __future__ import print_function
from __future__ import division

from keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Conv2D, Flatten
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input




#Loading and opening files
trainfile= '/Users/kodi/Downloads/food-101/meta/train.txt'
testfile = '/Users/kodi/Downloads/food-101/meta/test.txt'
or_fpath= '/Users/kodi/Downloads/food-101/images/'
trainpath = '/Users/kodi/Downloads/food-101/train/'
testpath = '/Users/kodi/Downloads/food-101/test/'

#CREATING TEST/TRAIN VARIABLES,
traintestfile = [trainfile,testfile]
newdata = []
for file in traintestfile:
    with open(file) as textfile:
        data= textfile.read() 
    newdata.append(str(data).split('\n') )

train = newdata[0]
test = newdata[1]

#INITIALIZE MODEL INPUTS
num_classes= len(classes)  
dropout = .5

batch_size = 32
dimensions = [256,256]
shape = (256,256,3)
epochs = 50
earlystop = EarlyStopping(monitor = 'accuracy', min_delta = 0, patience = 3, verbose = 1)
callbacks = [earlystop]


#TRAIN AND TEST DIRECTORIES\
maketesttraindir()

#makeclassdir([test,train])
makeclassdir(or_fpath)

#TRANFORM TO JPG'S FOR COMPARISON\
(trainimg, testimg) = tojpg([train, test])

#SENDS IMAGES INTO TRAIN AND TEST DIRECTORIES
images_to_train_test(or_fpath)

#RESIZE IMAGES\
resizev2([trainpath,testpath])

#REMOVE BAD FILES(REMEMBER TO TEST THIS!)
removebadfiles(trainpath)

#BEGIN TO SETUP MODEL
with open ('/Users/kodi/Downloads/food-101/meta/classes.txt') as ctgs:
    classes = ctgs.read().splitlines()
    



#RUN MODEL
train_gen, validation_gen = setup_generator(train_path, test_path, batch_size, dimensions) 
model_final = create_model(num_classes, dropout, shape = (256,256,3))
train_model(model_final, train_gen, validation_gen, callbacks)




In [18]:
#FUNCTIONS


#MAKE TEST/ TRAIN DIRECTORY\

def maketesttraindir():
    dirnametrain =  '/Users/kodi/Downloads/food-101/train/'
    dirnametest = '/Users/kodi/Downloads/food-101/test/'
    directories = [dirnametrain, dirnametest]
    for direc in directories:
        try:
            if not os.path.exists(direc):
                os.mkdir(direc)

        except:
            print('one already exists')



#CREATING CLASS DIRECTORIES
def makeclassdir(imgpath):
    try:
        for x in os.listdir(imgpath):

                if x.startswith('test'):

                        #.split('st')[1])
                        os.mkdir(testpath+ x.split('st')[1])
                else:
                    os.mkdir(trainpath+ x)
    except:
        pass

#CONVERT TEST/TRAIN TEXT FILES TO JPG IN ORDER TO FILTER AND PLACE THEM IN FOLDER
def tojpg(data):
    
    for test_train in data:
        test_train[:] =[file + '.jpg' for file in test_train]
    trainimg = data[0]
    testimg= data[1]
     #CHANGE LINES TO ONLY INCLUDE THE NUMBERS AND .JPG\
    #trainimg = [x.split('/')[1] for x in trainimg]    
    
    for idx,img in enumerate(trainimg):
        try:  
            trainimg[idx]= trainimg[idx].split('/')[1]
        except:
            pass
        
    return  trainimg ,testimg




#SEND IMAGES INTO DIRECTORIES

            
def images_to_train_test(imgpath):
    for filename in os.listdir(or_fpath):   
        try:   
                    for image in os.listdir(or_fpath + filename):

                            if filename.startswith('test'):
                                if os.path.exists('/Users/kodi/Downloads/food-101/'+ 'test/'+ filename.split('test')[1] + '/'+image):
                                    pass
                                else:  
                                    copy_tree('/Users/kodi/Downloads/food-101/images' + '/' +filename +'/','/Users/kodi/Downloads/food-101/'+ 'test/'+ filename.split('test')[1] + '/')                                                                                   
                            else:
                                if os.path.exists('/Users/kodi/Downloads/food-101/'+ 'train/'+ filename +'/'+image):
                                    pass
                                else:
                                    copy_tree('/Users/kodi/Downloads/food-101/images' + '/' +filename +'/','/Users/kodi/Downloads/food-101/'+ 'train/'+ filename + '/')                           
        except:
            pass






#RESIZE IMAGES

def resizev2(imgfolders):
    for fold in imgfolders:    
        for foodname in os.listdir(fold): 
            #print(fold + foodname)
            try:


                for item in os.listdir(fold + foodname):
                    #print(fold+foodname+'/'+item)

                          f_img = fold +foodname+"/"+item
                          img = Image.open(f_img)
                          img = img.resize((256,256))
                          img.save(f_img)

            

            except:
                pass
            
            
#REMOVE BAD IMAGE FILES           
            
def removebadfiles(trainpath):
    for foodname in os.listdir(trainpath):
        for image in os.listdir(trainpath + foodname):
            try:
                luv = Image.open(trainpath + foodname + '/'+image)
             
            except:
    
              os.remove(trainpath + foodname + '/'+image)

    
#BUIDLING IMAGE GENERATORS FOR TRAIN AND VALIDATION
    
def setup_generator(trainpath, testpath, batch_size, dimentions):
    from PIL import ImageFile
    ImageFile.LOAD_TRUNCATED_IMAGES = True

    train_datagen = ImageDataGenerator(
        rotation_range=30,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1. / 255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip = True,
        #preprocessing_function = True,
        #brightness_range = (.4,1.0),

        fill_mode='nearest')

    test_datagen = ImageDataGenerator(rescale=1. / 255)

    train_generator = train_datagen.flow_from_directory(
        trainpath,  # this is the target directory
        target_size=dimentions,
        class_mode="categorical",
        color_mode = 'rgb',
        batch_size= batch_size)

    validation_generator = test_datagen.flow_from_directory(
        testpath, # this is the target directory,
        class_mode="categorical",
        color_mode = 'rgb',
        target_size=dimentions,
        batch_size=batch_size)

    return train_generator, validation_generator

#CREATE THE RESNET MODEL AND ADD AUGMENTATIONS

def create_model(num_classes, dropout, shape):
    base_model = ResNet50(
        weights='imagenet',
        include_top=False,
        input_tensor=Input(
            shape=shape))

    x = base_model.output
    #x = Conv2D(128, kernel_size=3, activation= 'relu', input_shape =shape )(x)
    x = GlobalAveragePooling2D()(x)
    #x = Flatten()(x)

    x = Dropout(dropout)(x)
    predictions = Dense(num_classes, activation='softmax')(x)

    model_final = Model(inputs=base_model.input, outputs=predictions)

    return model_final


#TRAIN THE MODEL 

def train_model(model_final, train_generator, validation_generator, callbacks):

    model_final.compile(
        loss='categorical_crossentropy',
        optimizer='adam',
        metrics=['accuracy'])

    model_final.fit_generator(train_generator, validation_data=validation_generator,
                              epochs=epochs, callbacks=callbacks,
                              steps_per_epoch=train_generator.samples/batch_size,
                              validation_steps=validation_generator.samples/                             
                              batch_size)








